In [2]:
from astroquery.simbad import Simbad

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

In [5]:
def ra_to_rad(ra):
    """ right ascension in hhmmss.s to radians """
    # RA is a string formatted as hhmmss.s
    hrs = float(ra[:2])
    mins = float(ra[2:4])
    secs = float(ra[4:])

    degree = hrs*(360/24.0) + mins*(360/(24*60.0)) + secs*(360/(24*3600.0))
    rad = degree*(np.pi/180.0)
    return rad

def dec_to_rad(dec):
    """ declination in damas to radians
    dec is a string formatted as (+/-)ddamas
    """
    sign = 1.0 if dec[0] == '+' else -1.0
    degs = float(dec[1:3])
    mins = float(dec[3:5])
    secs = float(dec[5:])

    degree = sign * (degs + mins/60.0 + secs/3600.0)
    rad = degree * (np.pi/180.0)
    return rad

In [6]:
Simbad.list_votable_fields()

--NOTES--

1. The parameter filtername must correspond to an existing filter. Filters include: B,V,R,I,J,K.  They are checked by SIMBAD but not astroquery.simbad

2. Fields beginning with rvz display the data as it is in the database. Fields beginning with rv force the display as a radial velocity. Fields beginning with z force the display as a redshift

3. For each measurement catalog, the VOTable contains all fields of the first measurement. When applicable, the first measurement is the mean one. 

Available VOTABLE fields:

bibcodelist(y1-y2)
biblio
cel
cl.g
coo(opt)
coo_bibcode
coo_err_angle
coo_err_maja
coo_err_mina
coo_qual
coo_wavelength
coordinates
dec(opt)
dec_prec
diameter
dim
dim_angle
dim_bibcode
dim_incl
dim_majaxis
dim_minaxis
dim_qual
dim_wavelength
dimensions
distance
distance_result
einstein
fe_h
flux(filtername)
flux_bibcode(filtername)
flux_error(filtername)
flux_name(filtername)
flux_qual(filtername)
flux_system(filtername)
flux_unit(filtername)
fluxdata(filtername)

In [7]:
Simbad.get_field_description('v*')

variable stars parameters extracted mainly from the 
              General Catalog of Variable Stars by Kukarkin et al.
              USSR Academy of Sciences (3rd edition in 1969,and continuations)


In [8]:
Simbad.reset_votable_fields()
Simbad.add_votable_fields(
    'distance', 'dim', 'z_value', 'otypes',
    'flux(V)', 'flux(B)'
)
Simbad.get_votable_fields()

['main_id',
 'coordinates',
 'distance',
 'dim',
 'z_value',
 'otypes',
 'flux(V)',
 'flux(B)']

In [13]:
Simbad.ROW_LIMIT = 10000
Simbad.TIMEOUT = 180
south = Simbad.query_criteria('maintypes = * & Vmag < 7 & ra < 180').to_pandas()
north = Simbad.query_criteria('maintypes = * & Vmag < 7 & ra > 180').to_pandas()
# stars = Simbad.query_criteria('maintypes = * & Vmag < 6').to_pandas()
# res = Simbad.query_object('RS Pupus')

In [15]:
df = north.append(south)

df = df.rename(columns={'MAIN_ID': 'id'})
df.head()

,id,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,...,Distance_perr,Distance_method,Distance_bibcode,GALDIM_MAJAXIS,GALDIM_MINAXIS,GALDIM_ANGLE,Z_VALUE,OTYPES,FLUX_V,FLUX_B
0,b'HD 200253',21 01 12.8644,+36 01 33.647,14,14,0.0308,0.0386,90,A,O,...,1.5912,paral,2018yCat.1345....0G,NaN,NaN,0,-0.000029,b'*|IR',5.997,7.00
1,b'* 61 Cyg B',21 06 55.2640,+38 44 31.362,14,14,0.0437,0.0481,90,A,O,...,0.0007,paral,2018yCat.1345....0G,NaN,NaN,0,-0.000214,b'**|*|PM*|PM*|V*|Fl*|Ro*|IR|X',6.030,7.40
2,b'* tau Cyg',21 14 47.4925,+38 02 43.144,9,9,2.6400,2.6400,90,A,O,...,NaN,,,0.015,0.0,213,-0.000070,b'**|*|PM*|PM*|V*|dS*|IR|UV|UV|X',3.730,4.12
3,b'* tau Cyg A',21 14 47.5031,+38 02 43.610,14,14,0.3419,0.4304,90,A,O,...,0.1885,paral,2018yCat.1345....0G,NaN,NaN,0,NaN,b'**|*|PM*',3.830,4.23
4,b'* tau Cyg B',21 14 47.4627,+38 02 44.140,10,10,34.0000,34.0000,90,B,O,...,NaN,,,NaN,NaN,0,NaN,b'**|*|PM*',6.570,7.50


In [16]:
len(df)

14194

In [17]:
df.to_json('stars.json', orient='records')

In [ ]:
Simbad.ROW_LIMIT = 12000
cepheids = Simbad.query_criteria('maintype = cC*')

In [ ]:
df = cepheids.to_pandas()
df.head()

In [ ]:
df.to_json('cepheids.json', orient='records')

In [ ]:
df.columns

In [ ]:
len(df['Distance_distance'].unique())

In [ ]:
s = b'Pup'
df.query('@s in MAIN_ID')

In [ ]:
df[[(b'V*' in s['MAIN_ID'] and pd.notna(s['Distance_distance'])) for s in df.items()]]

In [ ]:
# has a distance
cleaned = df[pd.notna(df['Distance_distance'])]
# has a V__period
cleaned = cleaned[pd.notna(cleaned['V__period'])]
# has V flux
cleaned = cleaned[pd.notna(cleaned['FLUX_V'])]
# period based off V flux
cleaned = cleaned[[p == 'V' for p in cleaned['V__magtyp']]]
# distance isn't from Cepheid
cleaned = cleaned[[s != 'Cep' for s in cleaned['Distance_method']]]

# remove random star with period of 3212.24026...
cleaned = cleaned[[p < 100 for p in cleaned['V__period']]]

In [14]:
def abs_from_app_mag(apparent_magnitude, distance):
    # m - M = 5*log(dist / 10)
    #=> M = m - 5*log(dist/10)
    return apparent_magnitude - 5.0*np.log10(distance / 10.0)

def calc_mag(star, band='FLUX_V'):
    app_mag = star[band]
    dist = star['Distance_distance']
    if star['Distance_unit'] == 'kpc':
        dist *= 1000
    return abs_from_app_mag(app_mag, dist)

def calc_luminosity(star, band='FLUX_V'):
    # L = 4pi(flux)(dist^2)
    flux = star[band]
    dist = star['Distance_distance']
    if star['Distance_unit'] == 'kpc':
        dist *= 1000
    return 4.0*np.pi*flux*(dist**2)


def cartesian(star):
    ra = star['RA']
    dec = star['DEC']
    dist = star['Distance_distance']
    dist *= units.pc if star['Distance_unit'] == 'pc' else units.kpc
    unit = (units.hourangle, units.deg)
    sc = SkyCoord(ra + ' ' + dec, unit=unit, distance=dist)
    return sc.cartesian

In [ ]:
abs_mags = []
periods = []
for i,s in cleaned.iterrows():
    abs_mags.append(calc_mag(s))
    periods.append(s['V__period'])

In [ ]:
ax = plt.subplot(1, 1, 1, xscale='log')
plt.plot(periods, abs_mags, 'o')

In [ ]:
cleaned['V__magtyp'].unique()

In [ ]:
print(cleaned['Distance_distance'][21], cleaned['FLUX_V'][21])